In [70]:
import os
import requests
import json
import requests
import os
from urllib.parse import urlparse

In [75]:
destination_folder = "../data/downloaded"  # Folder where you want to save the image

# How many images for each psecies
count_per_species = 1000

# Specify the species name
species_name = "Impatiens glandulifera"

In [76]:
def save_image_from_url(url, id):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            file_path = os.path.join(destination_folder, f"{id}.jpg")
            with open(file_path, 'wb') as f:
                f.write(response.content)
            # print(f"Image saved as: {file_path}")
            return True
        else:
            print("Failed to download image: HTTP status code", response.status_code)
            return False
    except Exception as e:
        print("An error occurred:", str(e))
        return False

In [77]:
def fetch_photos_for_species(species_name):
    # Check the destination folder for existing files
    existing_files = [f for f in os.listdir(destination_folder) if os.path.isfile(os.path.join(destination_folder, f))]
    existing_files_count = len(existing_files)
    print(f"Existing files in the folder: {existing_files_count}")

    # Proceed with fetching photos only if the existing files count is less than the desired count_per_species
    remaining_count = count_per_species - existing_files_count
    if remaining_count <= 0:
        print("Desired image count already reached. No need to fetch more.")
        return
    elif remaining_count < count_per_species:
        print(f"Need to fetch {remaining_count} more images.")

    api_url = f"https://api.inaturalist.org/v1/observations?q={species_name}"
    total_images_saved = 0

    try:
        page = 1
        while total_images_saved < count_per_species:
            response = requests.get(api_url + f"&page={page}")
            print('page')
            print(page)
            
            if response.status_code == 200:
                data = response.json()
                results = data.get('results', [])
                if not results:
                    break
                
                for observation in results:
                    observation_id = observation['id']
                    if observation.get('observation_photos'):
                        for obs_photo in observation['observation_photos']:
                            photo_id = obs_photo['photo']['id'] 
                            photo_url = obs_photo['photo']['url']
                            if photo_url:
                                saved = save_image_from_url(photo_url, photo_id)
                                if saved:
                                    total_images_saved += 1
                                    if total_images_saved >= count_per_species:
                                        break
                            else:
                                print(f"No photo URL found for photo ID: {photo_id}")
                        if total_images_saved >= count_per_species:
                            break
                        for ex_photo in observation.get('photos', []):
                            ex_photo_id = ex_photo['id'] 
                            ex_photo_url = ex_photo['url']
                            if ex_photo_url:
                                saved = save_image_from_url(ex_photo_url, ex_photo_id)
                                if saved:
                                    total_images_saved += 1
                                    if total_images_saved >= count_per_species:
                                        break
                            else:
                                print(f"No photo URL found for photo ID: {photo_id}")
                        if total_images_saved >= count_per_species:
                            break
                    else:
                        print(f"No photos found for observation ID: {observation_id}")
                    if total_images_saved >= count_per_species:
                        break
                page += 1
            else:
                print(f"Error fetching data from iNaturalist API. Status code: {response.status_code}")
                break

        print(f"Total unique images saved: {total_images_saved}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [78]:
# Fetch and download photos for the specified species
fetch_photos_for_species(species_name)

Existing files in the folder: 1301
Desired image count already reached. No need to fetch more.
